# Phase Plane Position
Find different degrees of freedom for a single phase plane with a light field propagating through it.

Model should simulate:
* Light propagating from the start point to the phase mask ($z_1$).
* Light's phase is altered as it interacts with the phase mask.
* Light propagating from the phase mask to the end point ($z_2$).

From this model, a NN should be able to find out the distances $z_1$ and $z_2$ (when $z = z_1 + z_2$ is known). This is the "axial distance".

After this is reliably found, try changing the degree of freedom to lateral distance of the phase mask. Then increase the degree of freedom by adding another lateral axis (x and y axes).

Then try finding the correct alignment of all the parameters when they are initially unknown.

In [28]:
# Imports
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
import matplotlib.ticker as ticker

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import random

In [101]:
torch.manual_seed(0)

### Global Functions

In [135]:
# Make a vectorising function
def vectorise(field):
    x = field.shape[0]
    y = field.shape[1]
    return field.reshape(x*y,1)

# Make an unvectorising function
def unvectorise(field, dim):
    return field.reshape(dim,dim)

#def speckle_pattern():

def random_pi():
    """Generates a random number (3dp) between -pi and pi."""
    return random.randrange(-3141,3141,1)/1000

def random_phase_grid(size, reproduce=False, seed=0):
    """Generates a random square plane of specified size with each pixel having a random phase change attached to it.
    Ranges from -pi to pi. Can reproduce the same matrix if 'reproduce' is set to True."""
    if reproduce == 1:
        torch.manual_seed(seed)
        return 2 * np.pi * torch.rand(size,size) - np.pi
    elif reproduce == 0:
        return 2 * np.pi * torch.rand(size,size) - np.pi
    else:
        print(f"Invalid reproducability value given. Accepts either 'True' or 'False'. What was given: {reproduce}.")

def loss_function(output_fields, target_fields):
    similarity = 1
    for (n, output_field) in enumerate(output_fields):
        inner_product = (output_field * torch.conj(target_fields[n])).abs()
        similarity = similarity * inner_product.sum()
    return (1-similarity)

In [163]:
random_phase_grid(31).shape

torch.Size([31, 31])

### Classes

In [108]:
class Field(torch.Tensor):
    """Class designed around a 2D electric field represented by a torch tensor."""
    wl = 633e-9 #[m]
    k = torch.tensor(2*np.pi/wl) #[1/m]
    k_x = 1235678
    k_y = 3457437
    k_z = (torch.abs(k)**2 - k_x**2 - k_y**2)**0.5 #[1/m]
    
    def __init__(self, tensor):
        self = tensor

    def normalise(self):
        return self / torch.linalg.matrix_norm(self)

    def visualise(self, title=""):
        """Displays a visual plot of the field, using hsv colour mapping to demonstrate the 
        fields phase (Hue) and amplitude (Value)."""
        # Set up plots
        fig, axs = plt.subplots(1,2, figsize=(10,5))

        # Plot the given field
        axs[0].imshow(Complex2HSV(self, 0, 0.065))

        # Colour bar
        V, H = np.mgrid[0:1:100j, 0:1:300j]
        S = np.ones_like(V)
        HSV = np.dstack((H,S,V))
        RGB = hsv_to_rgb(HSV)

        axs[1].imshow(RGB, origin="lower", extent=[0, 2*np.pi, 0, 1], aspect=15)

        axs[1].set_xticks([0, np.pi, 2*np.pi], ['0', '$\pi$', '$2\pi$'])
        axs[1].set_yticks([0, 1], ['0', '1'])

        axs[1].set_ylabel("Normalised Amplitude")
        axs[1].set_xlabel("Phase (rad.)")

        fig.show()
        
        def propagate_along_z(input_field, z):
            """Propagates an initial electric field over a specified distance of free space. Assuming the direction of travel
            is along the z axis."""
            # Assuming each pixel is an individual plane wave:
            # Find the phase accumulated for each pixel over the propagation distance and add it on
            
            # Fourier transform a vectorised input electric field into momentum space.
            initial_p_field = torch.fft.fft(input_field)
            # Add on any extra phase the field accumulates.
            propagated_p_field = initial_p_field * torch.exp(1j*k_z*z)
            # Inverse fourier transform back into real space
            propagated_r_field = torch.fft.ifft(propagated_p_field)
            
            return propagated_r_field

### Code a Phase Mask

In [27]:
# Should be the size of the electric field, perhaps with a speckle pattern on it

### Axial distance